In [1]:
import argparse
import os
import time
import datetime
import torch
import torch.optim as optim
from math import ceil
from torch.utils.data import DataLoader
from dataset.dataset import CityScapesDataset
from utils.metric import runningScore, averageMeter
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F


os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
torch.cuda.is_available()

True

In [3]:
BASE_CHANNELS = 9
# n_train=2436
n_train=2476
n_val=400

batchsize   = 20
Epoch       = 50
img_size    = [256, 512]
model_name  = 'model_task2'
task        = 'cat'
lr = 3.75e-3

final_save_path='./'+model_name+'.pth'

In [4]:
# LOSS FUNCTIONS: CE + Dice + Lovasz-Softmax
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, pred, target):
        pred = F.softmax(pred, dim=1)
        num_classes = pred.shape[1]
        target_one_hot = F.one_hot(target, num_classes=num_classes)  # (B, H, W, C)
        target_one_hot = target_one_hot.permute(0, 3, 1, 2).float()  # (B, C, H, W)
        
        pred = pred.view(pred.shape[0], pred.shape[1], -1)  # (B, C, H*W)
        target_one_hot = target_one_hot.view(target_one_hot.shape[0], target_one_hot.shape[1], -1)
        
        intersection = (pred * target_one_hot).sum(dim=2)  # (B, C)
        union = pred.sum(dim=2) + target_one_hot.sum(dim=2)  # (B, C)
        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        
        return 1.0 - dice.mean()


class LovaszSoftmaxLoss(nn.Module):
    def __init__(self, ignore_index=255):
        super(LovaszSoftmaxLoss, self).__init__()
        self.ignore_index = ignore_index

    def forward(self, pred, target):
        pred = F.softmax(pred, dim=1)
        B, C, H, W = pred.shape
        pred = pred.permute(0, 2, 3, 1).reshape(-1, C)  # (B*H*W, C)
        target = target.view(-1)  # (B*H*W,)
        
        if self.ignore_index is not None:
            valid_mask = target != self.ignore_index
            pred = pred[valid_mask]
            target = target[valid_mask]
        
        if pred.numel() == 0:
            return pred.sum() * 0.0
        
        loss = self.lovasz_softmax_flat(pred, target, C)
        return loss

    def lovasz_softmax_flat(self, pred, target, num_classes):
        losses = []
        for c in range(num_classes):
            target_c = (target == c).float()
            pred_c = pred[:, c]
            errors = (target_c - pred_c).abs()
            errors_sorted, perm = torch.sort(errors, descending=True)
            target_sorted = target_c[perm]
            
            inter = target_sorted.sum() - target_sorted.cumsum(0)
            union = target_sorted.sum() + (1.0 - target_sorted).cumsum(0)
            iou = 1.0 - inter / union
            
            if len(errors_sorted) > 1:
                grad = iou[1:] - iou[:-1]
                loss_c = torch.dot(F.relu(errors_sorted[1:]), grad)
            else:
                loss_c = errors_sorted[0] if len(errors_sorted) > 0 else torch.tensor(0.0).to(pred.device)
            
            losses.append(loss_c)
        
        return torch.stack(losses).mean()

class ComposedLoss(nn.Module):
    def __init__(self, ce_weight=1.0, dice_weight=1.0, lovasz_weight=0.5):
        super(ComposedLoss, self).__init__()
        self.ce_weight = ce_weight
        self.dice_weight = dice_weight
        self.lovasz_weight = lovasz_weight
        
        self.ce_loss = nn.CrossEntropyLoss()
        self.dice_loss = DiceLoss()
        self.lovasz_loss = LovaszSoftmaxLoss()

    def forward(self, pred, target):
        ce = self.ce_loss(pred, target)
        dice = self.dice_loss(pred, target)
        lovasz = self.lovasz_loss(pred, target)
        
        total_loss = (self.ce_weight * ce +
                     self.dice_weight * dice +
                     self.lovasz_weight * lovasz)
        
        return total_loss

In [5]:
def train(epoch, data_loader, Net, optimizer, loss_fn, Meter):
    Net.train()
    timeStart = time.time()
    with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{Epoch}', unit='img') as pbar:
        for i, (data, target) in enumerate(data_loader):
            data , target = data.to(device),target.to(device)
            ### by yourself         
            optimizer.zero_grad()
            pred = Net(data)
            loss = loss_fn(pred, target)
            loss.backward()
            optimizer.step()



            ### End
            training_loss = loss.item()
            pbar.set_postfix(**{'loss (batch)': training_loss})
            pred = pred.data.max(1)[1]
            Meter['metric'].update(target.data.cpu().numpy(), pred.data.cpu().numpy())
            Meter['loss'].update(training_loss,data.size()[0])
            pbar.update(data.shape[0])
    timeEnd = time.time()
    score, class_iou = Meter['metric'].get_scores()
    loss_avg = Meter['loss'].avg
    print('epoch %3d : %10s loss: %f OverallAcc: %f MeanAcc %f mIoU %f time: %f' 
        %(epoch, ('training'), loss_avg, score['OverallAcc'], score['MeanAcc'], score['mIoU'], timeEnd-timeStart))

    return score['mIoU'],loss_avg

In [6]:
def val(epoch, data_loader, Net, loss_fn, Meter):
    Net.eval()
    with torch.no_grad():
        for i, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            timeStart = time.time()
            ### by yourself
            pred = Net(data)
            validation_loss = loss_fn(pred, target).item()
            timeEnd = time.time()           
            
            pred = pred.data.max(1)[1]
            Meter['metric'].update(target.data.cpu().numpy(), pred.data.cpu().numpy())
            Meter['loss'].update(validation_loss,data.size()[0])
            Meter['time'].update(timeEnd-timeStart,1)
    score, class_iou = Meter['metric'].get_scores()
    loss_avg = Meter['loss'].avg
    time_avg = Meter['time'].avg
    print('epoch %3d : %10s loss: %f OverallAcc: %f MeanAcc %f mIoU %f time: %f' 
        %(epoch, ('validation'), loss_avg, score['OverallAcc'], score['MeanAcc'], score['mIoU'], time_avg))
    
    return score['mIoU']

In [7]:
best_val_miou=0
current_val_miou=0

In [8]:
import network2
import copy
#from network2 import *
if __name__ == '__main__':

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   
    assert task in ['cat'],'wrong value of task'
    if task=='cat':
        num_classes = 8
        
    training_meter = {'metric':runningScore(num_classes),'loss':averageMeter(),'time':averageMeter()}
    validation_meter = {'metric':runningScore(num_classes),'loss':averageMeter(),'time':averageMeter()}

    print(str(datetime.datetime.now()))
    print('batchsize %3d | epoch %3d | img_size  %4d %4d | task  %6s | model_name  %25s '
                    %(batchsize,Epoch,img_size[0],img_size[1], task ,model_name ))

    TrainingDataset   = CityScapesDataset("/export/home/dl2025f/shared/data", "training", img_size, task=task, augmentation=None)
    ValidationDataset = CityScapesDataset("/export/home/dl2025f/shared/data", "validation", img_size, task=task)

    TrainingLoader    = DataLoader(TrainingDataset, batch_size=batchsize, shuffle=True, num_workers=4)
    ValidationLoader  = DataLoader(ValidationDataset, batch_size=batchsize, shuffle=False, num_workers=4)
    num_batch         = ceil(len(TrainingDataset)/batchsize)

    # define yout model
    Net = network2.EfficientUNet(n_channels=3, n_classes=num_classes, base_channels=BASE_CHANNELS)
    Net = Net.to(device)
    # define your optimizer
    optimizer = optim.Adam(Net.parameters(), lr=lr)
    # schduler
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
    # define your loss    
    loss_fn = ComposedLoss(ce_weight=1.0, dice_weight=1.0, lovasz_weight=0.5)

    start_epoch = 0
    for epoch in range(start_epoch, Epoch):
        for _, v in training_meter.items():
            v.reset()
            
        show_lr=optimizer.param_groups[0]['lr']
        print('learning rate is : ',  show_lr   )
        current_train_miou,current_train_loss=train(epoch, TrainingLoader, Net, optimizer, loss_fn, training_meter)
        
        scheduler.step() ###  StepLR
        
        if (epoch+1)%1==0 or epoch==Epoch-1:
            for _, v in validation_meter.items():
                v.reset()
            current_val_miou=val(epoch, ValidationLoader, Net, loss_fn, validation_meter)
            
            if current_val_miou>best_val_miou:
                best_val_miou=current_val_miou
                best_state_dict = Net.state_dict()
                torch.save(best_state_dict, final_save_path)
                print("(model saved)")

    print(str(datetime.datetime.now()))


2025-11-27 17:31:27.102597
batchsize  20 | epoch  50 | img_size   256  512 | task     cat | model_name                model_task2 
learning rate is :  0.00375


Epoch 1/50: 100%|██████████| 2476/2476 [01:53<00:00, 21.75img/s, loss (batch)=1.88]

epoch   0 :   training loss: 2.554615 OverallAcc: 0.556749 MeanAcc 0.266091 mIoU 0.182644 time: 113.840778


epoch   0 : validation loss: 1.861406 OverallAcc: 0.713980 MeanAcc 0.403972 mIoU 0.303115 time: 0.027552
(model saved)
learning rate is :  0.00375


Epoch 2/50: 100%|██████████| 2476/2476 [01:50<00:00, 22.42img/s, loss (batch)=1.44]

...

epoch  48 :   training loss: 0.781022 OverallAcc: 0.903168 MeanAcc 0.800189 mIoU 0.707052 time: 110.029022


epoch  48 : validation loss: 0.910306 OverallAcc: 0.877850 MeanAcc 0.783174 mIoU 0.676546 time: 0.027512
learning rate is :  0.0009375


Epoch 50/50: 100%|██████████| 2476/2476 [01:50<00:00, 22.44img/s, loss (batch)=0.891]

epoch  49 :   training loss: 0.781975 OverallAcc: 0.903004 MeanAcc 0.800897 mIoU 0.706767 time: 110.352105


epoch  49 : validation loss: 0.894663 OverallAcc: 0.883547 MeanAcc 0.778772 mIoU 0.680351 time: 0.027475
2025-11-27 19:18:40.758429


In [9]:
print(' best_val_miou is :  ',best_val_miou)

 best_val_miou is :   0.68088921440897


In [10]:
# INT8 QAT CONFIGURATION
QAT_EPOCHS = 35
QAT_LR = lr * 0.1
QAT_BACKEND = 'qnnpack'

In [11]:
# INT8 QAT TRAINING AND CONVERSION

# Step 1: Load baseline FP32 model
Net_qat = network2.load_model(final_save_path, base_channels=BASE_CHANNELS)
Net_qat = Net_qat.to(device)

# Step 2: Prepare model for QAT
print(f"\n[2/5] Preparing model for QAT (adding fake quantization)...")
Net_qat = network2.prepare_qat_model(Net_qat, backend=QAT_BACKEND)
Net_qat = Net_qat.to(device)

# Step 3: QAT Training (fine-tune with fake quantization)
print(f"\n[3/5] QAT Training for {QAT_EPOCHS} epochs...")
optimizer_qat = optim.Adam(Net_qat.parameters(), lr=QAT_LR)
scheduler_qat = optim.lr_scheduler.StepLR(optimizer_qat, step_size=5, gamma=0.5)

best_qat_miou = 0

for epoch in range(QAT_EPOCHS):
    # Training
    for _, v in training_meter.items():
        v.reset()
    
    current_train_miou_qat, current_train_loss_qat = train(
        epoch, TrainingLoader, Net_qat, optimizer_qat, loss_fn, training_meter
    )
    scheduler_qat.step()
    
    # Validation
    for _, v in validation_meter.items():
        v.reset()
    current_val_miou_qat = val(epoch, ValidationLoader, Net_qat, loss_fn, validation_meter)
    
    # Track best
    if current_val_miou_qat > best_qat_miou:
        best_qat_miou = current_val_miou_qat
        print(f"  (best QAT mIoU updated: {best_qat_miou:.6f})")

print(f"\nQAT Training Complete")
print(f"  Best QAT mIoU: {best_qat_miou:.6f}")

# Step 4: Convert to INT8 (moves model to CPU)
print(f"\n[4/5] Converting to INT8 quantized model...")
Net_qat = Net_qat.to('cpu')
Net_qat.eval()
Net_quantized = network2.convert_to_quantized(Net_qat)

# Step 5: Final evaluation of INT8 model on CPU
print(f"\n[5/5] Evaluating INT8 model on CPU...")
print(f"  Moving validation data to CPU for INT8 evaluation...")

# Create CPU validation function for quantized model
def val_cpu(epoch, data_loader, Net, loss_fn, Meter):
    Net.eval()
    with torch.no_grad():
        for i, (data, target) in enumerate(data_loader):
            # Keep data on CPU for quantized model
            data_cpu, target_cpu = data.cpu(), target.cpu()
            
            pred = Net(data_cpu)
            validation_loss = loss_fn(pred, target_cpu).item()
            
            pred = pred.data.max(1)[1]
            Meter['metric'].update(target_cpu.data.cpu().numpy(), pred.data.cpu().numpy())
            Meter['loss'].update(validation_loss, data_cpu.size()[0])
    
    score, class_iou = Meter['metric'].get_scores()
    loss_avg = Meter['loss'].avg
    print('epoch %3d : %10s loss: %f OverallAcc: %f MeanAcc %f mIoU %f (CPU)' 
        %(epoch, ('validation'), loss_avg, score['OverallAcc'], score['MeanAcc'], score['mIoU']))
    
    return score['mIoU']

for _, v in validation_meter.items():
    v.reset()
final_quantized_miou = val_cpu(-1, ValidationLoader, Net_quantized, loss_fn, validation_meter)

print(f"\n" + "="*80)
print(f"INT8 QUANTIZATION COMPLETE")
print(f"="*80)
print(f"  Final INT8 mIoU: {final_quantized_miou:.6f}")
print(f"  Threshold: 0.66")
if final_quantized_miou >= 0.66:
    print(f"Requirement satisfied (margin: +{final_quantized_miou - 0.66:.6f})")
else:
    print(f"Below threshold (shortfall: {0.66 - final_quantized_miou:.6f})")

# Save INT8 model (CPU model)
torch.save(Net_quantized.state_dict(), './model_task2_int8.pth')
print(f"\nINT8 model saved to: model_task2_int8.pth (CPU model)")

[load_model] Detected FP32 model
[load_model] Loaded FP32 model

[2/5] Preparing model for QAT (adding fake quantization)...
Model prepared for QAT with backend=qnnpack
Fake quantization modules added for training

[3/5] QAT Training for 35 epochs...


Epoch 1/50: 100%|██████████| 2476/2476 [01:50<00:00, 22.39img/s, loss (batch)=0.883]

epoch   0 :   training loss: 0.843997 OverallAcc: 0.891507 MeanAcc 0.784149 mIoU 0.687124 time: 110.579932


epoch   0 : validation loss: 0.935212 OverallAcc: 0.875061 MeanAcc 0.771773 mIoU 0.667671 time: 0.042677
  (best QAT mIoU updated: 0.667671)


Epoch 2/50: 100%|██████████| 2476/2476 [01:50<00:00, 22.44img/s, loss (batch)=0.808]

...

epoch  33 :   training loss: 0.804630 OverallAcc: 0.898534 MeanAcc 0.793979 mIoU 0.699631 time: 110.701701


epoch  33 : validation loss: 0.919096 OverallAcc: 0.877872 MeanAcc 0.775293 mIoU 0.673826 time: 0.042510


Epoch 35/50: 100%|██████████| 2476/2476 [01:52<00:00, 22.10img/s, loss (batch)=0.784]

epoch  34 :   training loss: 0.806426 OverallAcc: 0.898278 MeanAcc 0.793378 mIoU 0.699266 time: 112.047472


epoch  34 : validation loss: 0.921617 OverallAcc: 0.877786 MeanAcc 0.772510 mIoU 0.673192 time: 0.042767

QAT Training Complete
  Best QAT mIoU: 0.675107

[4/5] Converting to INT8 quantized model...
Model converted to INT8 quantized format
All weights and activations are now 8-bit integers
Note: Quantized model is on CPU (qnnpack/fbgemm requirement)

[5/5] Evaluating INT8 model on CPU...
  Moving validation data to CPU for INT8 evaluation...
epoch  -1 : validation loss: 0.932147 OverallAcc: 0.876710 MeanAcc 0.771203 mIoU 0.668445 (CPU)

INT8 QUANTIZATION COMPLETE
  Final INT8 mIoU: 0.668445
  Threshold: 0.66
Requirement satisfied (margin: +0.008445)

INT8 model saved to: model_task2_int8.pth (CPU model)
